# The Analytics Edge of Retail Optimization

Both binary variables and non-linear functions


#gurobipy
The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

In [1]:
%pip install gurobipy 

Note: you may need to restart the kernel to use updated packages.


# load require packages

In [2]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import plotly.express as px

### Lists

In [3]:
# Be careful with Python indices! 
# The range function range(a,b) creates a range of integers starting at <a> but ending at <b-1>
weeks = list(range(157,170))
weeks.remove(164)
print("weeks (t) ", weeks)
# Also note here that the first element of a list has index 0. The second element has index 1, and so on... 
w1 = weeks[0] #denotes the number of the first week in our planning horizon
print(w1)
w2 = weeks[1] #denotes the number of the second week in our planning horizon
print(w2)

weeks (t)  [157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169]
157
158


### Parameters

In [4]:
intercept = 1.978242858
p_coeff = -2.809634145
p1_coeff = 0.963410728
p2_coeff = 0.759639170
season_coeff = {
    1: 0, 2: -0.562046910, 3: 0.087545274, 4: -0.402637480, 5: -0.027326010, 6: 0.004349885,
    7: -0.036102297, 8: -0.069280527, 9: 0.160276197, 10: 1.104208897, 11: 1.122711287,
    12: 1.176802194, 13: 0.947945548
}

In [5]:
# Here we create a dictionary that associates a season with each week in the planning horizon
season = {}
for w in weeks:
    season[w] = np.ceil((w % 52) / 4)

print(season)

{157: np.float64(1.0), 158: np.float64(1.0), 159: np.float64(1.0), 160: np.float64(1.0), 161: np.float64(2.0), 162: np.float64(2.0), 163: np.float64(2.0), 165: np.float64(3.0), 166: np.float64(3.0), 167: np.float64(3.0), 168: np.float64(3.0), 169: np.float64(4.0)}


## Model 1 - no price ladder or business rules
#### DECISION VARIABLES:  prices in weeks 157 – 169  and demand in weeks 157 – 169 
######  Prices; 𝑝_𝑡  for  t = 157, … , 169   and   𝑑_𝑡  for  t = 157, … , 169
######  Demand; follows linearly additive model:
#### MAXIMIZE 𝑅𝑒𝑣𝑒𝑛𝑢𝑒= 𝑝157𝑑157 + 𝑝158𝑑158 + 𝑝159𝑑159 + ... + 𝑝169𝑑169
#### Objective function,  sales revenue in next quarter (weeks 157 – 169)

1.   List item
2.   List item


#### SUBJECT TO CONSTRAINTS:
#### Demand is characterized by our linearly additive model:
#### 𝑑_𝑡 = 2181 − 2801*𝑝_𝑡 + 929*𝑝_(𝑡−1) + 728*𝑝_(𝑡−2) −555.430*𝑆𝑒𝑎𝑠𝑜𝑛2 +...+ 949.056*𝑆𝑒𝑎𝑠𝑜𝑛13


In [6]:
# Create Gurobi model object - repository for all objects to be used in the model
mod1 = gp.Model ("price_model_1")

Restricted license - for non-production use only - expires 2026-11-23


In [7]:
# Define decision variables
p = mod1.addVars(weeks, ub = 1)

In [8]:
p

{157: <gurobi.Var *Awaiting Model Update*>,
 158: <gurobi.Var *Awaiting Model Update*>,
 159: <gurobi.Var *Awaiting Model Update*>,
 160: <gurobi.Var *Awaiting Model Update*>,
 161: <gurobi.Var *Awaiting Model Update*>,
 162: <gurobi.Var *Awaiting Model Update*>,
 163: <gurobi.Var *Awaiting Model Update*>,
 165: <gurobi.Var *Awaiting Model Update*>,
 166: <gurobi.Var *Awaiting Model Update*>,
 167: <gurobi.Var *Awaiting Model Update*>,
 168: <gurobi.Var *Awaiting Model Update*>,
 169: <gurobi.Var *Awaiting Model Update*>}

In [11]:
# Set objective function
# Example first 2 lines fully written out, 
# Short cut for writing out each line
#  after the first 2 weeks, the remaining weeks[2:], are developed using the sum() and a for loop  (weeks 159 on...)
#    price at week(wx) * demand (from linear demand fucntion at weekx)
obj_fn = mod1.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) 
                               for w in weeks[2:] if w in season),
                          GRB.MAXIMIZE)

KeyError: 164

In [ ]:
mod1.optimize()

In [ ]:
p[157] # to see 1st prediction

In [ ]:
p[158] # to see 2nd prediction

In [ ]:
# to see all predictions and save to a dataframe
df1 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df1.loc[w,"price"] = p[w].x       
df1    

In [ ]:
fig = px.line(df1, x=df1.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 2 - no business rules + price ladder

In [ ]:
p_ladder = [1.00, 0.95, 0.85, 0.75, 0.60, 0.50] # updated price ladder per business rule

In [ ]:
# Create model object
mod2 = gp.Model ("price_model_2")

In [ ]:
# Define decision variables which includes ladder constraint
p = mod2.addVars(weeks)
x = mod2.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod2.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod2.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod2.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)

In [ ]:
mod2.optimize()

In [ ]:
df2 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df2.loc[w,"price"] = p[w].x       
df2    

In [ ]:
fig = px.line(df2, x=df2.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 3 - at most 4 promotions

In [ ]:
# Create model object
mod3 = gp.Model ("price_model_3")

In [ ]:
# Define decision variables
p = mod3.addVars(weeks)
x = mod3.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod3.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod3.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod3.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod3.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)

In [ ]:
mod3.optimize()

In [ ]:
df3 = pd.DataFrame(data = None, index = weeks, columns = ["price"])
for w in weeks:
    df3.loc[w,"price"] = p[w].x       
df3    

In [ ]:
fig = px.line(df3, x=df3.index, y='price', markers=True)
fig.update_layout(plot_bgcolor= "white", xaxis_title= "week")
fig.update_traces(line_color= "red")
fig.show()

## Model 4 - full model

In [ ]:
# Create model object
mod4 = gp.Model ("price_model_4")

In [ ]:
# Define decision variables
p = mod4.addVars(weeks)
x = mod4.addVars(weeks, p_ladder, vtype= GRB.BINARY)

In [ ]:
# Set objective function
obj_fn = mod4.setObjective(p[w1] * (intercept + p_coeff*p[w1] + p1_coeff*1 + p2_coeff*1 + season_coeff[season[w1]]) +
                           p[w2] * (intercept + p_coeff*p[w2] + p1_coeff*p[w1] + p2_coeff*1 + season_coeff[season[w2]]) +
                           sum(p[w] * (intercept + p_coeff*p[w] + p1_coeff*p[w-1] + p2_coeff*p[w-2] + season_coeff[season[w]]) for w in weeks[2:]),
                          GRB.MAXIMIZE)

In [ ]:
# Select price ladder value
constr_select_ladder = mod4.addConstrs(sum(x[w,k] for k in p_ladder) == 1 for w in weeks)
# Select price
constr_select_price = mod4.addConstrs(p[w] == sum(k * x[w,k] for k in p_ladder) for w in weeks)
# At most 4 promotions
constr_4_promo = mod4.addConstr(sum(x[w,k] for k in p_ladder[1:] for w in weeks) <= 4)
# No consecutive promotions
constr_no_consec_promo = mod4.addConstrs((sum(x[w,k] for k in p_ladder[1:]) + sum(x[w+1,k] for k in p_ladder[1:]) <= 1) for w in weeks[:-1])